In [1]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

model = AutoModelForCausalLM.from_pretrained( 
    MODEL_NAME,  
    device_map=device,  
    torch_dtype="auto",  
    trust_remote_code=True,  
) 

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset

ds = load_dataset("Yelp/yelp_review_full")

README.md:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [18]:
ds["train"][22493]

{'label': 0,
 'text': "I must put my 2 cents worth in about Vincents.  I personally will never go to Vincents again.  We went one new year's eve for a friend's husband birthday.  He is French and they are from Florida--so I was trying to make it a special occassion.   I remember very little about the meal because the service was HORRIBLE.  Red wine came after salads had been served.  A white wine sat on our table unopened until we were able to flag someone down--the serving of the meal was uneven at best and then servers got switched mid meal.  For a dinner that cost more than $500--I certainly expected more.  When I called to complain--their offer to me was a glass of champagne and a dessert--to make up for an evening that was suppose to be special on several accounts and turned out to be a disaster."}

In [46]:
generation_template = """<|system|>
You are an experienced customer service advisor who responds to customers' reviews.<|end|>
<|user|>
### TASK ###

Below you will find a Customer Review that was left on your company's website. It is very important to the company that you identify the good and bad points raised by this customer.

First, identify all the terms in the Customer Review the customer expresses an opinion about.
Second, use your knowledge as an experienced customer service advisor to write a reply to the Customer Review. For each positive opinion that the customer has, reinforce this positive opinion to show how great your company is. For each negative opinion, do your best to provide a commercial gesture or a helpful suggestion. For each neutral opinion, make a short comment that matches the customer's tone.
Finally, structure your analysis in the form of a JSON list. Include all the opinion terms that you found, and the justifications for each of them, and assign them an aspect category from the list of Aspect Categories given below.

### LIST OF ASPECT CATEGORIES ###

- food quality
- service general
- restaurant general
- ambience general
- food style_options
- restaurant miscellaneous
- food prices
- restaurant prices
- drinks quality
- drinks style_options
- location general
- drinks prices
- food general

### LIST OF SENTIMENTS ###

- positive
- negative
- neutral

### EXAMPLE ###

Customer Review: "With the exception of our lemon salad that had so much pepper on it that our eyes started watering, the food here was decent, not great."
Customer Service Advisor Reply: "I'm really sorry to hear that you had a bad experience with our lemon salad :( We will try to improve the food quality before you visit us again. Thanks for your feedback about the decent food also - we will do our best to turn that neutral opinion into a positve one. I'm sad that you thought our food was not great, so I promise I'll ask the chef to focus on the food quality and improve it a lot!"
Structured Analysis: {example_absa}

### INPUT ###

Customer Review: {review_text}
<|end|>
<|assistant|>
"""

In [48]:
example_absa = """[{ \"opinion term\": \"lemon salad\", \"aspect category\": \"food quality\", \"sentiment\": \"negative\", \"justification\": \"exception\" }, { \"opinion term\": \"food\", \"aspect category\": \"food quality\", \"sentiment\": \"neutral\", \"justification\": \"decent\" }, { \"opinion term\": \"food\", \"aspect category\": \"food quality\", \"sentiment\": \"negative\", \"justification\": \"not great\" }]"""

In [52]:
#example_review = "This place's burgers are the absolute best in town, and even though the service is incredibly slow I'd definitely come back - I want to try the tomato sauce that my friend had which looked delicious!"

example_review = ds["train"][22493]["text"]

example_review

"I must put my 2 cents worth in about Vincents.  I personally will never go to Vincents again.  We went one new year's eve for a friend's husband birthday.  He is French and they are from Florida--so I was trying to make it a special occassion.   I remember very little about the meal because the service was HORRIBLE.  Red wine came after salads had been served.  A white wine sat on our table unopened until we were able to flag someone down--the serving of the meal was uneven at best and then servers got switched mid meal.  For a dinner that cost more than $500--I certainly expected more.  When I called to complain--their offer to me was a glass of champagne and a dessert--to make up for an evening that was suppose to be special on several accounts and turned out to be a disaster."

In [49]:
generation_template.format(review_text=example_review, example_absa=example_absa)

'<|system|>\nYou are an experienced customer service advisor who responds to customers\' reviews.<|end|>\n<|user|>\n### TASK ###\n\nBelow you will find a Customer Review that was left on your company\'s website. It is very important to the company that you identify the good and bad points raised by this customer.\n\nFirst, identify all the terms in the Customer Review the customer expresses an opinion about.\nSecond, use your knowledge as an experienced customer service advisor to write a reply to the Customer Review. For each positive opinion that the customer has, reinforce this positive opinion to show how great your company is. For each negative opinion, do your best to provide a commercial gesture or a helpful suggestion. For each neutral opinion, make a short comment that matches the customer\'s tone.\nFinally, structure your analysis in the form of a JSON list. Include all the opinion terms that you found, and the justifications for each of them, and assign them an aspect catego

In [57]:
# toggle Return full text NO
# UPDATE - cant do max_length=None anymore??? workaround set max_new_tokens to large number
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=2000, return_full_text=False) 

result = pipe(generation_template.format(review_text=example_review, example_absa=example_absa))

print(result[0]['generated_text'])

 Customer Review: I must put my 2 cents worth in about Vincents. I personally will never go to Vincents again. We went one new year's eve for a friend's husband birthday. He is French and they are from Florida--so I was trying to make it a special occasion. I remember very little about the meal because the service was HORRIBLE. Red wine came after salads had been served. A white wine sat on our table unopened until we were able to flag someone down--the serving of the meal was uneven at best and then servers got switched mid meal. For a dinner that cost more than $500--I certainly expected more. When I called to complain--their offer to me was a glass of champagne and a dessert--to make up for an evening that was suppose to be special on several accounts and turned out to be a disaster.

Structured Analysis:

[{ "opinion term": "Vincents", "aspect category": "restaurant general", "sentiment": "negative", "justification": "personal decision not to return" },
{ "opinion term": "service",

In [58]:
short_generation_template = """<|system|>
You are an experienced customer service advisor who responds to customers' reviews.<|end|>
<|user|>
### TASK ###

Below you will find a Customer Review that was left on your company's website. It is very important to the company that you identify the good and bad points raised by this customer.

First, identify all the terms in the Customer Review the customer expresses an opinion about.
Second, use your knowledge as an experienced customer service advisor to write a reply to the Customer Review. For each positive opinion that the customer has, reinforce this positive opinion to show how great your company is. For each negative opinion, do your best to provide a commercial gesture or a helpful suggestion. For each neutral opinion, make a short comment that matches the customer's tone.

### EXAMPLE ###

Customer Review: "With the exception of our lemon salad that had so much pepper on it that our eyes started watering, the food here was decent, not great."
Customer Service Advisor Reply: "I'm really sorry to hear that you had a bad experience with our lemon salad :( We will try to improve the food quality before you visit us again. Thanks for your feedback about the decent food also - we will do our best to turn that neutral opinion into a positve one. I'm sad that you thought our food was not great, so I promise I'll ask the chef to focus on the food quality and improve it a lot!"

### INPUT ###

Customer Review: {review_text}
<|end|>
<|assistant|>
"""

In [59]:
short_result = pipe(short_generation_template.format(review_text=example_review))

print(short_result[0]['generated_text'])

 Customer Service Advisor Reply:

Dear valued customer,

Thank you for taking the time to share your experience at Vincent's. We are truly sorry to hear that your visit did not meet your expectations. We understand that the service you received was not up to par, and we sincerely apologize for the inconvenience caused.

We appreciate your feedback about the meal and the wine service. It is clear that we fell short in providing a memorable and enjoyable experience for you and your friend. We will take your comments to heart and work diligently to improve our service and ensure that our guests receive the exceptional dining experience they deserve.

We understand that you had a special occasion, and we regret that it did not turn out as planned. We would like to offer you a complimentary dinner for two at our restaurant, with a personalized menu to ensure that your next visit is a delightful one.

Once again, we apologize for the disappointing experience and hope that we will have the op